In [1]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib.ticker import MultipleLocator

plt.rcParams["font.family"] = "Freesentation"
plt.rcParams["axes.unicode_minus"] = False


# 이상치 탐지

## 목표
- MAUDE 데이터에서 defect_type(키워드) 이상치 탐지하기
- 최근 달에 갑자기 증가한 키워드 찾기



In [2]:
# 데이터 로드
lf = pl.scan_parquet("../data/gold/maude.parquet")

# 기본 정보 확인
total_rows = lf.select(pl.len()).collect().item()
print(f"전체 행 수: {total_rows:,}개")


전체 행 수: 509,192개


In [3]:
lf.collect_schema()

Schema([('mdr_report_key', Int32),
        ('adverse_event_flag', Boolean),
        ('product_problem_flag', Boolean),
        ('date_occurred', Date),
        ('date_received', Date),
        ('date_manufactured', Date),
        ('event_type', Categorical),
        ('previous_use_flag', Boolean),
        ('single_use_flag', Boolean),
        ('reprocessed_and_reused_flag', Boolean),
        ('product_problems', String),
        ('manufacturer_name', String),
        ('brand_name', String),
        ('model_number', String),
        ('udi_di', String),
        ('product_code', String),
        ('operator', Boolean),
        ('product_name', String),
        ('patient_age', Int64),
        ('mdr_text', String),
        ('outcome_L', Boolean),
        ('outcome_H', Boolean),
        ('outcome_S', Boolean),
        ('outcome_C', Boolean),
        ('outcome_R', Boolean),
        ('outcome_D', Boolean),
        ('outcome_O', Boolean),
        ('patient_harm', String),
        ('problem_compo

# 이건 일단 넣어둬..

## 1. 날짜 처리

date_received 컬럼에서 월 정보를 추출


In [ ]:
def extract_month_from_date(lf, date_column):
    """
    날짜 컬럼에서 월 정보를 추출하는 함수
    
    Parameters:
    lf: polars LazyFrame
    date_column: 날짜 컬럼 이름 (예: 'date_received')
    
    Returns:
    월 정보가 추가된 LazyFrame (as_of_month 컬럼 추가)
    """
    result = lf.with_columns([
        pl.col(date_column).cast(pl.Date).dt.strftime("%Y-%m").alias("as_of_month")
    ])
    return result


In [ ]:
# 월 정보 추출
lf_with_month = extract_month_from_date(lf, "date_received")

# 확인
sample = lf_with_month.select(["date_received", "as_of_month"]).head(10).collect()
print("월 정보 추출 확인:")
print(sample.to_pandas())


## 2) 키워드 테이블 만들기

defect_type을 키워드로 사용


In [ ]:
def create_keyword_table(lf, keyword_column, month_column):
    """
    키워드 테이블 생성 함수 (report-level presence)
    defect_type의 각 값을 개별 keyword로 처리
    
    Parameters:
    lf: polars LazyFrame
    keyword_column: 키워드 컬럼 이름 (예: 'defect_type')
    month_column: 월 컬럼 이름 (예: 'as_of_month')
    
    Returns:
    키워드 테이블 LazyFrame (mdr_report_key, keyword, month)
    """
    result = (
        lf
        .filter(pl.col(keyword_column).is_not_null())
        .select([
            pl.col("mdr_report_key"),
            pl.col(keyword_column).alias("keyword"),
            pl.col(month_column).alias("month")
        ])
        .unique(subset=["mdr_report_key", "keyword", "month"])
    )
    return result

In [ ]:
# 키워드 테이블 생성
lf_keyword_table = create_keyword_table(lf_with_month, "defect_type", "as_of_month")

print("\n키워드 테이블 샘플:")
keyword_sample = lf_keyword_table.collect()
print(keyword_sample.to_pandas())

## 4. 월별 집계

두 가지를 집계해야 함:
1. 월별 키워드별 보고서 수 (C) 
2. 월별 전체 보고서 수 (N) 

In [ ]:
def monthly_keyword_counts(lf_keyword_table, month_column):
    """
    월별 키워드별 보고서 수 집계 함수
    
    Parameters:
    lf_keyword: 키워드 테이블 LazyFrame (keyword, month 포함)
    month_column: 월 컬럼 이름 (예: 'month')
    
    Returns:
    월별 키워드별 집계 결과 LazyFrame
    - keyword: 키워드
    - month: 월
    - n_reports_keyword: 그 달에 그 키워드를 포함한 보고서 수
    """
    result = (
        lf_keyword_table
        .group_by(["keyword", month_column])
        .agg([
            pl.col("mdr_report_key").n_unique().alias("n_reports_keyword")
        ])
        .sort(["keyword", month_column])
    )
    return result


In [ ]:
# 월별 키워드별 집계
lf_keyword_monthly = monthly_keyword_counts(lf_keyword_table, "month")

print("월별 키워드별 집계 결과 (샘플):")
keyword_monthly_sample = lf_keyword_monthly.head(20).collect()
print(keyword_monthly_sample.to_pandas())


In [ ]:
def monthly_total_reports(lf, date_column):
    """
    월별 전체 보고서 수 집계 함수
    
    Parameters:
    lf: 원본 LazyFrame
    date_column: 날짜 컬럼 이름 (예: 'date_received')
    
    Returns:
    월별 전체 보고서 수 LazyFrame
    - month: 월
    - n_total_reports: 그 달 전체 보고서 수
    """
    result = (
        lf
        .filter(pl.col(date_column).is_not_null())
        .with_columns([
            pl.col(date_column).cast(pl.Date).dt.strftime("%Y-%m").alias("month")
        ])
        .group_by("month")
        .agg([
            pl.len().alias("n_total_reports")
        ])
        .sort("month")
    )
    return result


In [ ]:
# 월별 전체 보고서 수 집계
lf_monthly_total = monthly_total_reports(lf, "date_received")

print("월별 전체 보고서 수:")
monthly_total = lf_monthly_total.collect()
print(monthly_total.to_pandas())


In [ ]:
def calculate_keyword_ratio_time_series(lf_keyword_monthly, lf_monthly_total, month_column):
    """
    키워드별 비율 시계열 계산 함수
    
    Parameters:
    lf_keyword_monthly: 월별 키워드별 집계 LazyFrame
    lf_monthly_total: 월별 전체 보고서 수 LazyFrame
    month_column: 월 컬럼 이름 (예: 'month')
    
    Returns:
    비율 시계열 LazyFrame
    - keyword: 키워드
    - month: 월
    - n_reports_keyword: 그 달에 그 키워드를 포함한 보고서 수
    - n_total_reports: 그 달 전체 보고서 수
    - p_t: 비율 (n_reports_keyword / n_total_reports)
    """
    result = (
        lf_keyword_monthly
        .join(lf_monthly_total, on=month_column, how="left")
        .with_columns([
            (pl.col("n_reports_keyword") / pl.col("n_total_reports")).alias("p_t")
        ])
        .sort(["keyword", month_column])
    )
    return result


In [ ]:
# 비율 시계열 계산
lf_keyword_ratio = calculate_keyword_ratio_time_series(lf_keyword_monthly, lf_monthly_total, "month")

print("키워드별 비율 시계열 (샘플):")
keyword_ratio_sample = lf_keyword_ratio.head(20).collect()
print(keyword_ratio_sample.to_pandas())


## 3) 베이스라인 비교를 위한 윈도우 설정

### 개념 정리 (헷갈려서 다시 정리)
- **최근 윈도우**: 지금 상황 (최근 1개월 or 3개월)
- **기준 윈도우**: 과거 비교 대상 (직전 1개월 or 3개월)

윈도우 구성 :

    - 최근 윈도우 구성 
        - 1개월 : 최근 완료된 1개월 (ex. 2025-11)
        - 3개월 : 최근 완료된 3개월 (ex. 2025-11, 10, 9)

    - 기준 윈도우
        - 직전 1월 / 직전 3개월 (ex. 2025-10 / 10, 9, 8)
    
    - 슬라이딩 offset은 1개월로 지정

    - `C_recent` = 최근기간 키워드 보고서 합
    - `C_base` = 기준기간 키워드 보고서 합      
    - `N_recent` = 최근기간 전체 보고서 합
    - `N_base` = 기준기간 전체 보고서 합

In [ ]:
def create_window_aggregates_by_keyword(lf_keyword_monthly, lf_monthly_total, month_column, window_size):
    """
    키워드별 윈도우 집계 함수 (최근 윈도우와 기준 윈도우)
    create_complete_baseline_table과 동일한 윈도우 계산 방식 및 데이터 소스 사용
    
    Parameters:
    lf_keyword_monthly: 키워드별 월별 집계 LazyFrame (keyword, month, n_reports_keyword)
    lf_monthly_total: 월별 전체 보고서 수 LazyFrame
    month_column: 월 컬럼 이름
    window_size: 윈도우 크기 (1 또는 3)
    
    Returns:
    키워드별 윈도우 집계 결과 LazyFrame
    - keyword: 키워드
    - C_recent: 최근 k개월의 n_reports_keyword 합
    - N_recent: 최근 k개월의 n_total_reports 합 (키워드 무관, 전체 보고서 수)
    - C_base: 직전 k개월의 n_reports_keyword 합
    - N_base: 직전 k개월의 n_total_reports 합 (키워드 무관, 전체 보고서 수)
    """
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    
    # 각 키워드별로 처리
    # create_complete_baseline_table과 동일한 데이터 소스 사용
    df_keyword_monthly = lf_keyword_monthly.collect()
    df_monthly_total = lf_monthly_total.collect()
    
    # 전체 보고서 수에서 최근/기준 윈도우 계산 (키워드와 무관)
    df_monthly_sorted = df_monthly_total.sort(month_column, descending=True)
    
    if len(df_monthly_sorted) < window_size * 2:
        return pl.DataFrame({
            'keyword': [],
            'C_recent': [],
            'N_recent': [],
            'C_base': [],
            'N_base': []
        }).lazy()
    
    # create_complete_baseline_table과 동일한 방식으로 최신 월 가져오기
    # Polars DataFrame에서 최신 월 가져오기
    as_of_month = df_monthly_sorted.select(month_column).to_series()[0]
    as_of_date = datetime.strptime(as_of_month, "%Y-%m")
    
    # 최근 기간 계산 (create_complete_baseline_table과 동일)
    if window_size == 1:
        recent_months = [as_of_month]
        base_months = [(as_of_date - relativedelta(months=1)).strftime("%Y-%m")]
    else:  # window_size == 3
        recent_months = [
            (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
            for i in range(3)
        ]
        base_months = [
            (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
            for i in range(1, 4)
        ]
    
    # 전체 보고서 수 집계 (키워드와 무관)
    # create_complete_baseline_table과 동일한 방식으로 집계
    recent_months_data = df_monthly_sorted.filter(pl.col(month_column).is_in(recent_months))
    N_recent = recent_months_data.select(pl.col('n_total_reports').sum()).item()
    
    base_months_data = df_monthly_sorted.filter(pl.col(month_column).is_in(base_months))
    N_base = base_months_data.select(pl.col('n_total_reports').sum()).item()
    
    results = []
    
    for keyword in df_keyword_monthly['keyword'].unique():
        keyword_data = (
            df_keyword_monthly
            .filter(pl.col("keyword") == keyword)
            .sort(month_column, descending=True)
        )
        
        # 최근 윈도우: recent_months에 해당하는 데이터
        recent_data = keyword_data.filter(pl.col(month_column).is_in(recent_months))
        C_recent = recent_data['n_reports_keyword'].sum() if len(recent_data) > 0 else 0
        
        # 기준 윈도우: base_months에 해당하는 데이터
        base_data = keyword_data.filter(pl.col(month_column).is_in(base_months))
        C_base = base_data['n_reports_keyword'].sum() if len(base_data) > 0 else 0
        
        results.append({
            'keyword': keyword,
            'C_recent': C_recent,
            'N_recent': N_recent,  # 키워드와 무관한 전체 보고서 수
            'C_base': C_base,
            'N_base': N_base  # 키워드와 무관한 전체 보고서 수
        })
    
    # 결과를 LazyFrame으로 변환
    if results:
        result = pl.DataFrame(results).lazy()
    else:
        result = pl.DataFrame({
            'keyword': [],
            'C_recent': [],
            'N_recent': [],
            'C_base': [],
            'N_base': []
        }).lazy()
    
    return result


In [ ]:
# 1개월 윈도우 집계 (키워드별)
# create_complete_baseline_table과 동일한 데이터 소스 사용
lf_window_1m = create_window_aggregates_by_keyword(lf_keyword_monthly, lf_monthly_total, "month", 1)

print("1개월 윈도우 집계 (키워드별):")
window_1m = lf_window_1m.collect()
print(window_1m.to_pandas())


In [ ]:
# # 윈도우 계산 확인 (디버깅)
# print("=" * 60)
# print("윈도우 계산 확인")
# print("=" * 60)

# # 데이터의 최신 월 확인
# latest_months = lf_monthly_total.sort("month", descending=True).head(6).collect()
# print("\n데이터의 최신 6개월:")
# print(latest_months.to_pandas())

# print("\n" + "=" * 60)


In [ ]:
# 3개월 윈도우 집계 (키워드별)
# create_complete_baseline_table과 동일한 데이터 소스 사용
lf_window_3m = create_window_aggregates_by_keyword(lf_keyword_monthly, lf_monthly_total, "month", 3)

print("3개월 윈도우 집계 (키워드별):")
window_3m = lf_window_3m.collect()
print(window_3m.to_pandas())


## 4) 절대 빈도 고려 및 비율 계산

- 하드 가드: C_recent >= 20
- 비율 계산: ratio, p_recent, p_base, score_ratio


In [ ]:
def ratio_score(lf):
    """
    비율과 점수를 계산하는 함수
    
    Parameters:
    lf: 윈도우 집계 결과 LazyFrame (C_recent, N_recent, C_base, N_base 포함)
    
    Returns:
    비율과 점수가 추가된 LazyFrame
    - ratio: 스무딩 포함 비율 (C_recent + 1) / (C_base + 1)
    - p_recent: C_recent / N_recent
    - p_base: C_base / N_base
    - score_ratio: log(ratio) * log(C_recent + 1)
    """
    # 먼저 ratio를 계산
    result = lf.with_columns([
        # 스무딩 포함 비율
        ((pl.col("C_recent") + 1) / (pl.col("C_base") + 1)).round(2).alias("ratio"),
        # p_recent와 p_base
        (pl.col("C_recent") / pl.col("N_recent")).round(2).alias("p_recent"),
        (pl.col("C_base") / pl.col("N_base")).round(2).alias("p_base")
    ])
    
    # ratio가 계산된 후에 score_ratio 계산 (polars에서 이미 계산된 컬럼 참조)
    result = result.with_columns([
        # 가중 점수 계산: log(ratio) * log(C_recent + 1)
        # polars에서 log 계산 (ratio + 1e-10으로 0 방지)
        ((pl.col("ratio")).log() * (pl.col("C_recent") + 1).log()).round(2).alias("score_ratio")
    ])
    
    return result


In [ ]:
# 1개월 윈도우 비율 및 점수 계산
lf_window_1m_with_score = ratio_score(lf_window_1m)
print("1개월 윈도우 비율 및 점수:")
window_1m_with_score = lf_window_1m_with_score.collect()
print(window_1m_with_score.to_pandas())


In [ ]:
# 3개월 윈도우 비율 및 점수 계산
lf_window_3m_with_score = ratio_score(lf_window_3m)

print("3개월 윈도우 비율 및 점수:")
window_3m_with_score = lf_window_3m_with_score.collect()
print(window_3m_with_score.to_pandas())


In [ ]:
def frequency_guard(lf, min_count=20):
    """
    절대 빈도 가드를 적용하는 함수 (C_recent >= min_count)
    
    Parameters:
    lf: 윈도우 집계 결과 LazyFrame
    min_count: 최소 건수 기준 (기본값 20)
    
    Returns:
    가드를 통과한 결과만 필터링된 LazyFrame
    """
    result = lf.filter(pl.col("C_recent") >= min_count)
    return result


In [ ]:
# 절대 빈도 가드 적용 (1개월)
lf_filtered_1m = frequency_guard(lf_window_1m_with_score, min_count=20)

print("1개월 윈도우 - 가드 통과 결과:")
filtered_1m = lf_filtered_1m.collect()
print(filtered_1m.to_pandas())


In [ ]:
# 절대 빈도 가드 적용 (3개월)
lf_filtered_3m = frequency_guard(lf_window_3m_with_score, min_count=20)

print("3개월 윈도우 - 가드 통과 결과:")
filtered_3m = lf_filtered_3m.collect()
print(filtered_3m.to_pandas())


## 5) 상대적 증가율 탐지

ratio >= 2.0 조건을 만족하는 이상치 탐지


In [ ]:
def detect_spike_by_ratio(lf, ratio_threshold=2.0):
    """
    상대적 증가율로 이상치를 탐지하는 함수
    
    Parameters:
    lf: 가드를 통과한 윈도우 집계 결과 LazyFrame
    ratio_threshold: 비율 임계값 (기본값 2.0)
    기본값 2.0 / 보수적인 값 3.0 (선택)
    
    Returns:
    이상치로 탐지된 결과만 필터링된 LazyFrame
    """
    result = lf.filter(pl.col("ratio") >= ratio_threshold)
    return result


In [ ]:
def run_spike_detection(lf_filtered, window_name, ratio_threshold=1.5):
    """
    이상치 탐지를 실행하고 결과를 출력하는 함수
    
    Parameters:
    lf_filtered: 필터링된 LazyFrame
    window_name: 윈도우 이름 (예: "1개월", "3개월")
    ratio_threshold: ratio 기준값
    
    Returns:
    탐지 결과 DataFrame
    """
    lf_spike = detect_spike_by_ratio(lf_filtered, ratio_threshold=ratio_threshold)
    spike_result = lf_spike.collect()
    
    print(f"\n{window_name} 윈도우 이상치 탐지 결과:")
    print(f"이상치로 탐지된 키워드 수: {len(spike_result)}")
    
    if len(spike_result) > 0:
        print(spike_result.to_pandas())
    else:
        print("이상치로 탐지된 키워드가 없습니다.")
    
    return spike_result

# 실행
spike_1m = run_spike_detection(lf_filtered_1m, "1개월")
spike_3m = run_spike_detection(lf_filtered_3m, "3개월")

# 총합본

## 6) 통합 베이스라인 집계 함수

모든 키워드를 포함하여 1개월/3개월 윈도우를 통합 계산하는 함수

In [4]:
def create_complete_baseline_table(
    lf,
    as_of_month=None,
    spike_threshold_ratio=None,
    min_c_recent=20
):
    """
    베이스라인 집계 테이블을 Long Format으로 생성하는 함수
    모든 keyword(defect_type)에 대해 0이어도 결과에 포함됩니다.
    
    Parameters:
    -----------
    lf_keyword_monthly : pl.LazyFrame
        키워드별 월별 집계 데이터 (keyword, month, n_reports_keyword)
    lf_monthly_total : pl.LazyFrame
        월별 전체 보고서 수 데이터 (month, n_total_reports)
    as_of_month : str, optional
        기준 월 (예: "2025-11"). None이면 자동으로 최신 월 사용
    spike_threshold_ratio : float, default=2.0
        스파이크 판단 기준 비율 (ratio >= spike_threshold_ratio)
    min_c_recent : int, default=20
        스파이크 판단 최소 C_recent 값 (C_recent >= min_c_recent)
    
    Returns:
    --------
    pl.LazyFrame
        Long Format 베이스라인 집계 테이블 (모든 keyword 포함)
    """
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    import time
    
    # 전체 실행 시간 측정 시작
    start_time = time.time()

    # 2. 월 정보 추출
    lf_with_month = lf.with_columns([
        pl.col("date_received").cast(pl.Date).dt.strftime("%Y-%m").alias("as_of_month")
    ])
    
    # 3. 키워드 테이블 생성
    lf_keyword_table = (
        lf_with_month
        .filter(pl.col("defect_type").is_not_null())
        .select([
            pl.col("mdr_report_key"),
            pl.col("defect_type").alias("keyword"),
            pl.col("as_of_month").alias("month")
        ])
        .unique(subset=["mdr_report_key", "keyword", "month"])
    )
    
    # 4. 월별 키워드별 집계
    lf_keyword_monthly = (
        lf_keyword_table
        .group_by(["keyword", "month"])
        .agg([
            pl.col("mdr_report_key").n_unique().alias("n_reports_keyword")
        ])
        .sort(["keyword", "month"])
    )
    
    # 5. 월별 전체 보고서 수 집계
    lf_monthly_total = (
        lf
        .filter(pl.col("date_received").is_not_null())
        .with_columns([
            pl.col("date_received").cast(pl.Date).dt.strftime("%Y-%m").alias("month")
        ])
        .group_by("month")
        .agg([
            pl.len().alias("n_total_reports")
        ])
        .sort("month")
    )
    
    # as_of_month가 지정되지 않으면 자동으로 최신 월 가져오기
    if as_of_month is None:
        available_months = (
            lf_monthly_total
            .select("month")
            .sort("month", descending=True)
            .collect()
            .to_series()
            .to_list()
        )
        as_of_month = available_months[0] if available_months else None
        if as_of_month is None:
            raise ValueError("사용 가능한 월 데이터가 없습니다.")
        print(f"자동으로 최신 월을 사용합니다: {as_of_month}")
    
    # as_of_month를 기준으로 기간 계산
    as_of_date = datetime.strptime(as_of_month, "%Y-%m")
    
    # 최근 기간 계산
    # ============================================================
    # 베이스라인 윈도우 구성 (상대 증가율 + 검정 공통)
    # ============================================================
    # 최근 윈도우 2종:
    # - 1개월: 최근 완료된 1개월 (예: 11월)
    recent_1month = [as_of_month]
    
    # - 3개월: 최근 완료된 3개월 (예: 11, 10, 9월)
    recent_3month = [
        (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
        for i in range(3)  # i=0: 11월, i=1: 10월, i=2: 9월
    ]
    
    # 기준 윈도우 (직전 기간, 겹치지 않도록):
    # - 직전 1개월: 최근 1개월의 직전 1개월 (예: 10월)
    base_1month = [(as_of_date - relativedelta(months=1)).strftime("%Y-%m")]
    
    # - 직전 3개월: 최근 3개월의 직전 3개월 (예: 10, 9, 8월)
    # recent_3month의 첫 번째 월(10월)부터 직전 3개월
    base_3month = [
        (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
        for i in range(1, 4)  # i=1: 10월, i=2: 9월, i=3: 8월
    ]
    
    # 슬라이딩 offset: 1개월 (최근 윈도우와 기준 윈도우 사이)
    # 키워드별 집계: C_recent, C_base, N_recent, N_base
    
    print(f"\n기준 월: {as_of_month}")
    print(f"최근 1개월: {recent_1month}")
    print(f"최근 3개월: {recent_3month}")
    print(f"기준 1개월: {base_1month}")
    print(f"기준 3개월: {base_3month}")
    
    # 모든 keyword 목록 가져오기 (0이어도 포함하기 위해)
    step0_start = time.time()
    all_keywords = (
        lf_keyword_monthly
        .select("keyword")
        .unique()
        .collect()
    )
    step0_time = time.time() - step0_start
    print(f"[시간 측정] 모든 keyword 목록 수집: {step0_time:.3f}초")
    print(f"총 keyword 개수: {len(all_keywords)}")
    
    # 1개월 윈도우: 키워드별 보고서 수 집계
    step1_start = time.time()
    baseline_1month_keyword_raw = (
        lf_keyword_monthly
        .filter(pl.col("month").is_in(recent_1month + base_1month))
        .with_columns([
            pl.when(pl.col("month").is_in(recent_1month))
            .then(pl.lit("recent"))
            .otherwise(pl.lit("base"))
            .alias("window_type")
        ])
        .group_by(["keyword", "window_type"])
        .agg([
            pl.col("n_reports_keyword").sum().alias("C")
        ])
        .collect()
        .pivot(
            index="keyword",
            on="window_type",
            values="C",
            aggregate_function="first"
        )
        .with_columns([
            pl.col("recent").fill_null(0).alias("C_recent_1m"),
            pl.col("base").fill_null(0).alias("C_base_1m")
        ])
        .select([
            pl.col("keyword"),
            pl.col("C_recent_1m"),
            pl.col("C_base_1m")
        ])
    )
    
    # 모든 keyword를 포함하도록 outer join
    baseline_1month_keyword = (
        all_keywords
        .join(
            baseline_1month_keyword_raw,
            on="keyword",
            how="left"
        )
        .with_columns([
            pl.col("C_recent_1m").fill_null(0),
            pl.col("C_base_1m").fill_null(0)
        ])
    )
    step1_time = time.time() - step1_start
    
    # 1개월 윈도우: 전체 보고서 수 집계
    step2_start = time.time()
    n_total_1month_reports = (
        lf_monthly_total
        .filter(pl.col("month").is_in(recent_1month + base_1month))
        .with_columns(
            pl.when(pl.col("month").is_in(recent_1month))
            .then(pl.lit('recent'))
            .otherwise(pl.lit('base'))
            .alias('period')
        )
        .group_by('period')
        .agg(pl.sum('n_total_reports').alias('N'))
        .collect()
        .to_pandas()
    )
    
    N_recent_1m = n_total_1month_reports[n_total_1month_reports['period'] == 'recent']['N'].values[0] if len(n_total_1month_reports[n_total_1month_reports['period'] == 'recent']) > 0 else 0
    N_base_1m = n_total_1month_reports[n_total_1month_reports['period'] == 'base']['N'].values[0] if len(n_total_1month_reports[n_total_1month_reports['period'] == 'base']) > 0 else 0
    step2_time = time.time() - step2_start
    
    # 3개월 윈도우: 키워드별 보고서 수 집계
    step3_start = time.time()

    # Recent 3개월 집계 (11, 10, 9월)
    recent_3m_keyword = (
        lf_keyword_monthly
        .filter(pl.col("month").is_in(recent_3month))
        .group_by("keyword")
        .agg([
            pl.col("n_reports_keyword").sum().alias("C_recent_3m")
        ])
        .collect()
    )

    # Base 3개월 집계 (10, 9, 8월)
    base_3m_keyword = (
        lf_keyword_monthly
        .filter(pl.col("month").is_in(base_3month))
        .group_by("keyword")
        .agg([
            pl.col("n_reports_keyword").sum().alias("C_base_3m")
        ])
        .collect()
    )

    # 두 결과 조인
    baseline_3month_keyword_raw = (
        recent_3m_keyword
        .join(base_3m_keyword, on="keyword", how="outer")
        .with_columns([
            pl.col("C_recent_3m").fill_null(0),
            pl.col("C_base_3m").fill_null(0)
        ])
    )

    # 모든 keyword를 포함하도록 outer join
    baseline_3month_keyword = (
        all_keywords
        .join(baseline_3month_keyword_raw, on="keyword", how="left")
        .with_columns([
            pl.col("C_recent_3m").fill_null(0),
            pl.col("C_base_3m").fill_null(0)
        ])
    )

    step3_time = time.time() - step3_start
    
    # # 디버깅: 특정 키워드의 base_3month 월별 합계 확인
    # test_keyword = "Electrical/Power"
    # test_keyword_monthly = (
    #     lf_keyword_monthly
    #     .filter((pl.col("keyword") == test_keyword) & (pl.col("month").is_in(base_3month)))
    #     .collect()
    # )
    # if len(test_keyword_monthly) > 0:
    #     manual_sum = test_keyword_monthly["n_reports_keyword"].sum()
    #     print(f"\n[디버깅] {test_keyword}의 base_3month ({base_3month}) 월별 데이터:")
    #     print(test_keyword_monthly.to_pandas())
    #     print(f"[디버깅] {test_keyword}의 base_3month 수동 합계: {manual_sum}")
    #     test_result = baseline_3month_keyword_raw.filter(pl.col("keyword") == test_keyword)
    #     if len(test_result) > 0:
    #         print(f"[디버깅] {test_keyword}의 C_base_3m (함수 결과): {test_result['C_base_3m'][0]}")
    
    # # 모든 keyword를 포함하도록 outer join
    # baseline_3month_keyword = (
    #     all_keywords
    #     .join(
    #         baseline_3month_keyword_raw,
    #         on="keyword",
    #         how="left"
    #     )
    #     .with_columns([
    #         pl.col("C_recent_3m").fill_null(0),
    #         pl.col("C_base_3m").fill_null(0)
    #     ])
    # )
    # step3_time = time.time() - step3_start
    
    # 3개월 윈도우: 전체 보고서 수 집계
    step4_start = time.time()

    # Recent 3개월 집계 (11, 10, 9월)
    N_recent_3m = (
        lf_monthly_total
        .filter(pl.col("month").is_in(recent_3month))
        .select(pl.sum('n_total_reports'))
        .collect()
        .item()
    )

    # Base 3개월 집계 (10, 9, 8월)
    N_base_3m = (
        lf_monthly_total
        .filter(pl.col("month").is_in(base_3month))
        .select(pl.sum('n_total_reports'))
        .collect()
        .item()
    )

    step4_time = time.time() - step4_start
    print(f"[시간 측정] 3개월 전체 보고서 집계: {step4_time:.3f}초")
    
    # 1개월 데이터 준비
    step5_start = time.time()
    baseline_1m = (
        baseline_1month_keyword
        .with_columns([
            pl.lit(as_of_month).alias("as_of_month"),
            pl.lit("1").alias("window"),
            pl.col("C_recent_1m").fill_null(0).cast(pl.Int64).alias("C_recent"),
            pl.col("C_base_1m").fill_null(0).cast(pl.Int64).alias("C_base"),
            pl.lit(N_recent_1m).cast(pl.Int64).alias("N_recent"),
            pl.lit(N_base_1m).cast(pl.Int64).alias("N_base")
        ])
        .select([
            "as_of_month",
            "window",
            "keyword",
            "C_recent",
            "C_base",
            "N_recent",
            "N_base"
        ])
    )
    
    # 3개월 데이터 준비
    baseline_3m = (
        baseline_3month_keyword
        .with_columns([
            pl.lit(as_of_month).alias("as_of_month"),
            pl.lit("3").alias("window"),
            pl.col("C_recent_3m").fill_null(0).cast(pl.Int64).alias("C_recent"),
            pl.col("C_base_3m").fill_null(0).cast(pl.Int64).alias("C_base"),
            pl.lit(N_recent_3m).cast(pl.Int64).alias("N_recent"),
            pl.lit(N_base_3m).cast(pl.Int64).alias("N_base")
        ])
        .select([
            "as_of_month",
            "window",
            "keyword",
            "C_recent",
            "C_base",
            "N_recent",
            "N_base"
        ])
    )
    step5_time = time.time() - step5_start
    
    # Long format으로 통합
    step6_start = time.time()
    
    # Long format으로 통합 및 계산
    threshold = spike_threshold_ratio
    
    # 1단계: ratio와 is_spike 계산
    # 스키마 통일을 위해 모든 숫자 컬럼을 Int64로 캐스팅
    baseline_1m = baseline_1m.with_columns([
        pl.col("C_recent").cast(pl.Int64),
        pl.col("C_base").cast(pl.Int64),
        pl.col("N_recent").cast(pl.Int64),
        pl.col("N_base").cast(pl.Int64)
    ])
    baseline_3m = baseline_3m.with_columns([
        pl.col("C_recent").cast(pl.Int64),
        pl.col("C_base").cast(pl.Int64),
        pl.col("N_recent").cast(pl.Int64),
        pl.col("N_base").cast(pl.Int64)
    ])
    
    baseline_final = (
        pl.concat([baseline_1m, baseline_3m])
        .with_columns([
            # 스무딩 포함 비율 계산: ratio = (C_recent + 1) / (C_base + 1)
            ((pl.col("C_recent") + 1) / (pl.col("C_base") + 1))
            .round(2)
            .alias("ratio"),
            # is_spike 계산 (필터 후보: ratio >= 2 또는 ratio >= 3)
            pl.when(
                (pl.col("C_recent") >= min_c_recent) & 
                (((pl.col("C_recent") + 1) / (pl.col("C_base") + 1)) >= threshold)
            )
            .then(True)
            .otherwise(False)
            .alias("is_spike")
        ])
    )
    
    # 2단계: ratio가 계산된 후에 가중 점수 계산
    baseline_final = (
        baseline_final
        .with_columns([
            # 소프트 가중(랭킹 반영) - 건수가 많을수록 우선순위
            # 방법 1: log(C_recent + 1) 사용
            (pl.col("ratio") * (pl.col("C_recent") + 1).log()).round(2).alias("score_log"),
            # 방법 2: sqrt(C_recent) 사용
            (pl.col("ratio") * pl.col("C_recent").sqrt()).round(2).alias("score_sqrt"),
            
            # 상대적 증가율 탐지(랭킹 1)
            # 가중 점수: score_ratio = log(ratio) * log(C_recent + 1)
            (pl.col("ratio").log() * (pl.col("C_recent") + 1).log()).round(2).alias("score_ratio"),
        ])
        .sort(["keyword", "window"])
    )
    
    step6_time = time.time() - step6_start
    total_time = time.time() - start_time
    
    print(f"[시간 측정] 전체 실행 시간: {total_time:.3f}초 ({total_time/60:.2f}분)")
    print(f"  - 모든 keyword 목록 수집: {step0_time:.3f}초 ({step0_time/total_time*100:.1f}%)")
    print(f"  - 1개월 키워드 집계: {step1_time:.3f}초 ({step1_time/total_time*100:.1f}%)")
    print(f"  - 1개월 전체 보고서 집계: {step2_time:.3f}초 ({step2_time/total_time*100:.1f}%)")
    print(f"  - 3개월 키워드 집계: {step3_time:.3f}초 ({step3_time/total_time*100:.1f}%)")
    print(f"  - 3개월 전체 보고서 집계: {step4_time:.3f}초 ({step4_time/total_time*100:.1f}%)")
    print(f"  - 데이터 준비 및 통합: {step5_time:.3f}초 ({step5_time/total_time*100:.1f}%)")
    print(f"  - 최종 통합 및 계산: {step6_time:.3f}초 ({step6_time/total_time*100:.1f}%)")
    
    # LazyFrame으로 변환하여 반환
    result = baseline_final.lazy()
    
    return result

In [5]:
# 통합 베이스라인 집계 함수 실행
lf_baseline_final = create_complete_baseline_table(
    lf,
    as_of_month='2025-11',  # None이면 자동으로 최신 월 사용
    spike_threshold_ratio=2.0
)

print("\n통합 베이스라인 집계 결과:")
baseline_final = lf_baseline_final.collect().to_pandas()
display(baseline_final)

# score_log  (소프트 가중 1 -> 건수가 많을수록 우선순위)
# score_sqrt (소프트 가중 2 -> 건수가 많을수록 우선순위)
# score_ratio (가중 점수 -> 배수증가, 최근 절대 건수 증가)


기준 월: 2025-11
최근 1개월: ['2025-11']
최근 3개월: ['2025-11', '2025-10', '2025-09']
기준 1개월: ['2025-10']
기준 3개월: ['2025-10', '2025-09', '2025-08']
[시간 측정] 모든 keyword 목록 수집: 0.102초
총 keyword 개수: 13
[시간 측정] 3개월 전체 보고서 집계: 0.036초
[시간 측정] 전체 실행 시간: 0.233초 (0.00분)
  - 모든 keyword 목록 수집: 0.102초 (43.6%)
  - 1개월 키워드 집계: 0.025초 (10.5%)
  - 1개월 전체 보고서 집계: 0.022초 (9.5%)
  - 3개월 키워드 집계: 0.047초 (20.0%)
  - 3개월 전체 보고서 집계: 0.036초 (15.3%)
  - 데이터 준비 및 통합: 0.001초 (0.4%)
  - 최종 통합 및 계산: 0.001초 (0.3%)

통합 베이스라인 집계 결과:


/var/folders/qk/10b0q2j971j6d2py2x39ys200000gn/T/ipykernel_50538/3421434177.py:237: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  recent_3m_keyword


,as_of_month,window,keyword,C_recent,C_base,N_recent,N_base,ratio,is_spike,score_log,score_sqrt,score_ratio
0,2025-11,1,Alarm/Alert,92,65,13241,11522,1.41,False,6.39,13.52,1.56
1,2025-11,3,Alarm/Alert,223,194,35641,36259,1.15,False,6.22,17.17,0.76
2,2025-11,1,Communication/Connectivity,354,328,13241,11522,1.08,False,6.34,20.32,0.45
3,2025-11,3,Communication/Connectivity,949,944,35641,36259,1.01,False,6.93,31.11,0.07
4,2025-11,1,Electrical/Power,1128,1012,13241,11522,1.11,False,7.80,37.28,0.73
5,2025-11,3,Electrical/Power,2804,2530,35641,36259,1.11,False,8.81,58.78,0.83
6,2025-11,1,Environmental/Compatibility,86,101,13241,11522,0.85,False,3.80,7.88,-0.73
7,2025-11,3,Environmental/Compatibility,288,325,35641,36259,0.89,False,5.04,15.10,-0.66
8,2025-11,1,Functional Failure,990,830,13241,11522,1.19,False,8.21,37.44,1.20
9,2025-11,3,Functional Failure,2637,2696,35641,36259,0.98,False,7.72,50.32,-0.16


## 그래프

In [ ]:
# 위에꺼 시각화 (키워드별 개별 그래프)

df_monthly = lf_keyword_monthly.collect().to_pandas()

# 년도와 월 분리
df_monthly['year'] = df_monthly['month'].str[:4]
df_monthly['month_num'] = df_monthly['month'].str[5:7].astype(int)

# 키워드 목록 가져오기 (정렬)
keywords = sorted(df_monthly['keyword'].unique())

# 각 키워드별로 서브플롯 생성 (세로로 배치)
fig, axes = plt.subplots(len(keywords), 1, figsize=(16, 5 * len(keywords)))

# 키워드가 1개인 경우 axes를 리스트로 변환
if len(keywords) == 1:
    axes = [axes]

# 년도별 색상
year_colors = {
    '2023': '#1f77b4',  # 파란색
    '2024': '#ff7f0e',  # 주황색
    '2025': '#2ca02c'   # 초록색
}

# 년도별 텍스트 오프셋 (데이터 포인트 숫자 겹침 방지)
offset_map = {
    '2023': 0,
    '2024': 6,
    '2025': 3
}

# 각 키워드별로 그래프 그리기
for idx, keyword in enumerate(keywords):
    ax = axes[idx]
    
    # 해당 키워드 데이터 필터링 및 정렬
    keyword_data = df_monthly[df_monthly['keyword'] == keyword].sort_values(['year', 'month_num'])
    
    # 년도별로 그리기
    years = sorted(keyword_data['year'].unique())
    
    for year in years:
        year_data = keyword_data[keyword_data['year'] == year].sort_values('month_num')
        color = year_colors.get(year, '#000000')
        
        # 선 그래프 그리기
        ax.plot(year_data['month_num'], year_data['n_reports_keyword'], 
                marker='o', linewidth=2.5, markersize=7, 
                label=f'{year}', color=color, alpha=0.8)
        
    # y축 범위 계산 및 설정
    y_min = keyword_data['n_reports_keyword'].min()
    y_max = keyword_data['n_reports_keyword'].max()
    
    # y축 범위를 정수로 조정 (약간의 여유 공간 포함)
    if y_max == y_min:
        # 값이 모두 같은 경우
        y_bottom = max(0, int(y_min) - 1)
        y_top = int(y_max) + 1
    else:
        y_range = y_max - y_min
        y_bottom = max(0, int(y_min - y_range * 0.1))
        y_top = int(y_max + y_range * 0.1) + 1
    
    ax.set_ylim(bottom=y_bottom, top=y_top)
    
    # y축 틱 설정: 정수 틱만 사용하고 중복 제거
    from matplotlib.ticker import MultipleLocator
    import numpy as np
    
    # 데이터 범위에 따라 적절한 간격 설정
    y_range = y_top - y_bottom
    if y_range <= 2:
        tick_interval = 1
    elif y_range <= 5:
        tick_interval = 1
    elif y_range <= 10:
        tick_interval = 2
    elif y_range <= 20:
        tick_interval = 5
    else:
        tick_interval = max(1, int(y_range / 6))
    
    # 정수 틱만 사용
    ax.yaxis.set_major_locator(MultipleLocator(tick_interval))
    # 포맷터: 정수로 표시
    ax.yaxis.set_major_formatter(lambda x, p: f'{int(x)}')
    
    # 각 서브플롯 설정
    ax.set_title(f'{keyword}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('보고서 수', fontsize=12)
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(['1월', '2월', '3월', '4월', '5월', '6월', 
                        '7월', '8월', '9월', '10월', '11월', '12월'])
    ax.legend(title='Year', fontsize=10, title_fontsize=11, loc='best')
    ax.grid(True, alpha=0.3)

    # # 정수로 변경 (y축)
    # ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
    # ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, pos: f'{int(x):d}'))
    
plt.tight_layout()
# plt.show()  # 필요시 주석 해제

In [ ]:
# 비율 시계열 그래프 (키워드별 개별 그래프)

df_ratio = lf_keyword_ratio.collect().to_pandas()

# 년도와 월 분리
df_ratio['year'] = df_ratio['month'].str[:4]
df_ratio['month_num'] = df_ratio['month'].str[5:7].astype(int)

# 키워드 목록 가져오기 (정렬)
keywords = sorted(df_ratio['keyword'].unique())

# 각 키워드별로 서브플롯 생성 (세로로 배치)
fig, axes = plt.subplots(len(keywords), 1, figsize=(16, 5 * len(keywords)))

# 키워드가 1개인 경우 axes를 리스트로 변환
if len(keywords) == 1:
    axes = [axes]

# 년도별 색상
year_colors = {
    '2023': '#1f77b4',  # 파란색
    '2024': '#ff7f0e',  # 주황색
    '2025': '#2ca02c'   # 초록색
}

# 년도별 텍스트 오프셋 (데이터 포인트 숫자 겹침 방지)
offset_map = {
    '2023': 0,
    '2024': 6,
    '2025': 3
}

# 각 키워드별로 그래프 그리기
for idx, keyword in enumerate(keywords):
    ax = axes[idx]
    
    # 해당 키워드 데이터 필터링 및 정렬
    keyword_data = df_ratio[df_ratio['keyword'] == keyword].sort_values(['year', 'month_num'])
    
    # 년도별로 그리기
    years = sorted(keyword_data['year'].unique())
    
    for year in years:
        year_data = keyword_data[keyword_data['year'] == year].sort_values('month_num')
        color = year_colors.get(year, '#000000')
        
        # 선 그래프 그리기 (비율을 퍼센트로 표시)
        ax.plot(year_data['month_num'], year_data['p_t'] * 100, 
                marker='o', linewidth=2.5, markersize=7, 
                label=f'{year}', color=color, alpha=0.8)
        
    # y축 범위 계산 및 설정 (비율이므로 0~100% 범위 고려)
    y_min = (keyword_data['p_t'] * 100).min()
    y_max = (keyword_data['p_t'] * 100).max()
    
    # y축 범위를 조정 (약간의 여유 공간 포함)
    if y_max == y_min:
        # 값이 모두 같은 경우
        y_bottom = max(0, y_min - 1)
        y_top = y_max + 1
    else:
        y_range = y_max - y_min
        y_bottom = max(0, y_min - y_range * 0.1)
        y_top = y_max + y_range * 0.1
    
    ax.set_ylim(bottom=y_bottom, top=y_top)
    
    # y축 틱 설정: 비율에 맞게 조정
    # 데이터 범위에 따라 적절한 간격 설정
    y_range = y_top - y_bottom
    if y_range <= 2:
        tick_interval = 0.5
    elif y_range <= 5:
        tick_interval = 1
    elif y_range <= 10:
        tick_interval = 2
    elif y_range <= 20:
        tick_interval = 5
    else:
        tick_interval = max(1, int(y_range / 6))
    
    ax.yaxis.set_major_locator(MultipleLocator(tick_interval))
    
    # 각 서브플롯 설정
    ax.set_title(f'{keyword} - 비율 시계열', fontsize=14, fontweight='bold')
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('비율 (%)', fontsize=12)
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(['1월', '2월', '3월', '4월', '5월', '6월', 
                        '7월', '8월', '9월', '10월', '11월', '12월'])
    ax.legend(title='Year', fontsize=10, title_fontsize=11, loc='best')
    ax.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.show()


## 최종 결과
result_1m = lf_filtered_1m.collect()

result_3m = lf_filtered_3m.collect()

## 컬럼 구성:
- keyword, C_recent, N_recent, C_base, N_base, ratio, p_recent, p_base, score_ratio